In [1]:
%reload_ext autoreload
%autoreload 2
import pickle
from utils import *

# Load Danmaku

In [2]:
recording_dir_list = [
    "/mnt/e/BiliRec_apixC/",
    "/mnt/e/ApixC_footages/",
    "/mnt/d/B站录播姬下载目录/tmp/",
]

In [28]:
danmaku_pool = DanmakuPool()

# traverse all xml files in dir and subdirs in recording_dir_list
def traverse_dir(dir):
    for file in os.listdir(dir):
        path = os.path.join(dir, file)
        if os.path.isdir(path):
            traverse_dir(path)
        elif file.endswith(".xml"):
            danmaku_pool.add_danmaku_from_xml(path, verbose=True)

for dir in recording_dir_list:
    traverse_dir(dir)

Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220528_044719_8点飞行棋联动.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220528_062942_8点飞行棋联动.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220528_063518_8点飞行棋联动.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220528_063634_8点飞行棋联动.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220528_073102_8点飞行棋联动.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220528_073134_8点飞行棋联动.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220620_042615_阿C又在玩什么.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220621_042741_MC 宝可梦之旅.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220621_080217_MC 宝可梦之旅.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220622_051108_我还是很聪明的对吧.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220623_043210_阿C又在玩什么.xml
Loaded danmaku from /mnt/e/BiliRec_apixC/saved/47867_20220623_043604_阿C又在玩什么.xml
Loaded danmaku from /mnt

In [29]:
# count duplicated danmaku
print("Found {} duplicated danmaku".format(danmaku_pool.df.duplicated(subset=["time", "roomid", "uid", "text"]).sum()))

Found 913 duplicated danmaku


In [36]:
# remove duplicated danmaku
danmaku_pool.df.drop_duplicates(inplace=True)

In [37]:
# save danmaku_pool to file
with open("cache/danmaku_pool.pickle", "wb") as f:
    pickle.dump(danmaku_pool, f)

In [3]:
# load danmaku_pool from file
with open("cache/danmaku_pool.pickle", "rb") as f:
    danmaku_pool = pickle.load(f)

# Validate Danmaku Pool

In [21]:
# detect wrong offset
danmaku_pool.df[(danmaku_pool.df["offset"] < -100) | (danmaku_pool.df["offset"] > 100)]

In [31]:
df = danmaku_pool.df[danmaku_pool.df.duplicated(subset=["time", "roomid", "uid", "text"], keep=False)].sort_values(by=["time"])
pairs = {}
for _, row in df.iterrows():
    id = (row["time"], row["uid"])
    if id in pairs:
        if row["source"] not in pairs[id]:
            pairs[id].append(row["source"])
    else:
        pairs[id] = [row["source"]]
print(set([tuple(pairs[key]) for key in pairs]))
